In [54]:
### STEP 1 --讀檔及載入套件--

# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso


data_path = './data/'
df_train = pd.read_csv(data_path + 'train_data.csv')
df_test = pd.read_csv(data_path + 'test_features.csv')



In [55]:
### STEP 2 --資料處理 產生初步的 TRAIN DATA 及 TEST DATA --

# poi : 直接轉TRUE 1 FALSE 0 -- TRUE FALSE 經過READ_CSV會被轉換成Boolean
#train_Y = df_train['poi']
train_Y = df_train['poi'].replace({True: 1, False: 0})
ids = df_test['name']
df_train_corr = df_train  #檢查相關係數用的TRAIN DATA
df_train = df_train.drop(['name', 'poi'] , axis=1)
df_test = df_test.drop(['name'] , axis=1)
test_X = df_test
#df = df_train
df = pd.concat([df_train,df_test])
df.head()

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,4.0,NaN,1617011.0,174839.0,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,14.0,NaN,NaN,1573324.0,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,NaN,NaN,602671.0,907502.0,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,25.0,NaN,375304.0,486.0,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [56]:
# 檢查 DataFrame 空缺值的狀態
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data.head(20))
na_check(df)

df.drop

# == practice == 把遺缺值七成以上的特徵排除
##nan_features = ('loan_advances','director_fees','restricted_stock_deferred','deferral_payments')
df = df.drop(['loan_advances', 'director_fees','restricted_stock_deferred','deferral_payments'] , axis=1)
df_train_corr = df_train_corr.drop(['loan_advances', 'director_fees','restricted_stock_deferred','deferral_payments'] , axis=1)

,Missing Ratio
loan_advances,97.260274
director_fees,88.356164
restricted_stock_deferred,87.671233
deferral_payments,73.287671
deferred_income,66.438356
long_term_incentive,54.794521
bonus,43.835616
from_messages,41.095890
from_poi_to_this_person,41.095890
from_this_person_to_poi,41.095890


In [57]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中 --目前除了姓名跟EMAIL 全數都是數值型
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
test_num = test_X.shape[0]


15 Numeric Features : ['bonus', 'deferred_income', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi', 'long_term_incentive', 'other', 'restricted_stock', 'salary', 'shared_receipt_with_poi', 'to_messages', 'total_payments', 'total_stock_value']



In [48]:
## 離群值處理 outlier -- 這部份我沒有處理 --
# 將 1stFlrSF 限制在你覺得適合的範圍內, 調整離群值
##"""
##Your Code Here
##"""

##df['1stFlrSF'] = df['1stFlrSF'][:train_num].clip(600, 1750)
##sns.regplot(x = df['1stFlrSF'][:train_num], y=train_Y)
##plt.show()

In [58]:
## 補空值的方法
df=df.fillna(0)
#df = df.fillna(-1)
#df = df.fillna(df.mean())

In [59]:
## 最大最小化 或 標準化處理
## https://www.itread01.com/content/1547461823.html

MMEncoder = MinMaxScaler()

## X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
## X_scaled = X_std * (max - min) + min

STDEncoder = StandardScaler()

##
##
df.head()

,bonus,deferred_income,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,long_term_incentive,other,restricted_stock,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,-3504386.0,19794175.0,46950.0,18.0,42.0,4.0,1617011.0,174839.0,2748364.0,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,0.0,19250000.0,29336.0,108.0,88.0,30.0,1920000.0,22122.0,6843672.0,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,-4167.0,1624396.0,22884.0,39.0,13.0,14.0,0.0,1573324.0,869220.0,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,0.0,0.0,118134.0,0.0,0.0,0.0,602671.0,907502.0,985032.0,224305.0,0.0,0.0,2652612.0,985032.0
4,1250000.0,-262500.0,0.0,35818.0,144.0,199.0,25.0,375304.0,486.0,126027.0,240189.0,2188.0,2598.0,1639297.0,126027.0


In [62]:
## PRACTICE 使用 CORR 相關係數，刪除特徵值 參考D40

## 觀察相關係數,列出目標 (TARGET) 與所有欄位之間相關係數，數值最大以及最小各 15 個
df_corr = df_train_corr.corr()
#print(df_corr.to_xarray()) 
print(df_corr['poi'].sort_values())

# max_corr = df_corr[1:12]
# min_corr = df_corr[-10:]
#print(max_corr)
##print(min_corr)

# 參考D30 篩選相關係數1

corr_features = list(df_corr[(df_corr['poi']>0.1) | (df_corr['poi']<-0.1)].index)
corr_features.remove('poi')

# 把遺缺值七成以上的特徵排除
##nan_features = ('loan_advances','director_fees','restricted_stock_deferred','deferral_payments')
##for i in nan_features:
##    corr_features.remove(i)

 
print(len(corr_features))
print(corr_features)
corr_features.pop(0)
print(type(corr_features))

L1_list = corr_features

###-not finish- df = df[corr_features]
###-not finish- train_X = MMEncoder.fit_transform(df[corr_features])
###-not finish- cross_val_score(estimator, train_X, train_Y, cv=5).mean()


deferred_income           -0.224654
from_messages             -0.126760
from_this_person_to_poi   -0.096864
to_messages               -0.023047
shared_receipt_with_poi    0.120467
from_poi_to_this_person    0.127932
expenses                   0.136386
other                      0.225619
long_term_incentive        0.227805
bonus                      0.242170
restricted_stock           0.261102
total_payments             0.271335
salary                     0.371917
total_stock_value          0.443098
exercised_stock_options    0.591972
poi                        1.000000
Name: poi, dtype: float64
13
['bonus', 'deferred_income', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'long_term_incentive', 'other', 'restricted_stock', 'salary', 'shared_receipt_with_poi', 'total_payments', 'total_stock_value']
<class 'list'>


In [29]:
## PRACTICE 使用LASSO 檢查共線性，刪除特徵值 參考D40
from sklearn.linear_model import Lasso
# 切分訓練集/測試集
##x_train, x_test, y_train, y_test = train_test_split(diabetes.data, diabetes.target, test_size=0.2, random_state=4)
train_X=df[:train_num]
test_X=df[:test_num]
# 建立一個線性回歸模型
lasso = Lasso(alpha=1000)

# 將訓練資料丟進去模型訓練
lasso.fit(train_X, train_Y)

# 將測試資料丟進模型得到預測結果
y_pred = lasso.predict(test_X)

# 檢查相關性
lasso.coef_

from itertools import compress
L1_mask = list((lasso.coef_>0) | (lasso.coef_<0))
print(len(L1_mask),type(L1_mask),L1_mask)
L1_list = list(compress(list(df), list(L1_mask)))  
L1_list.pop(0)

print(len(L1_list),type(L1_list),L1_list)

15 <class 'list'> [True, True, True, True, False, False, False, True, True, True, True, False, False, True, True]
9 <class 'list'> ['deferred_income', 'exercised_stock_options', 'expenses', 'long_term_incentive', 'other', 'restricted_stock', 'salary', 'total_payments', 'total_stock_value']


In [66]:
### PRACTICE 確認最後選完的特徵 ，確認 train_X test_X 是否適合  ###

train_X=df[:train_num]
test_X=df[:test_num]
train_X =  MMEncoder.fit_transform(train_X[L1_list])
test_X =  MMEncoder.fit_transform(test_X[L1_list])

print(df.shape)
print(train_num)
print(test_X.shape)
print(train_X.shape)
print(train_Y.shape)

(146, 15)
113
(33, 12)
(113, 12)
(113,)


In [67]:
print(train_X)


[[0.00000000e+00 5.76276747e-01 2.05234238e-01 ... 1.90854871e-01
  4.87689271e-03 4.59505908e-01]
 [1.00000000e+00 5.60433935e-01 1.28237521e-01 ... 4.51071350e-01
  8.38425392e-02 5.31750703e-01]
 [9.98810919e-01 4.72917736e-02 1.00033659e-01 ... 2.01016125e-02
  1.93500290e-02 5.16275414e-02]
 ...
 [1.00000000e+00 8.38409166e-04 1.38365907e-01 ... 1.60812900e-01
  1.33902353e-02 1.48290569e-03]
 [1.00000000e+00 0.00000000e+00 2.07638473e-03 ... 1.04042412e-01
  4.58672218e-06 4.60061060e-02]
 [1.00000000e+00 1.76962328e-02 6.02225884e-01 ... 1.00000000e+00
  1.02119555e-02 2.09767346e-02]]


In [31]:
#### PRACTICE 參數最佳化 ####
## 模型參數最佳化  D47
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

# 設定要訓練的超參數組合
n_estimators = [50, 100, 200, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

clf = GradientBoostingRegressor(random_state=7)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(train_X, train_Y)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

grid_result.best_params_

# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best Accuracy: -0.156196 using {'max_depth': 1, 'n_estimators': 50}


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    0.5s finished


In [32]:
### PRACTICE 確認最後選完的特徵 ，確認 train_X test_X 是否適合  ###

# 使用三種模型 : 邏輯斯迴歸 / 梯度提升機 / 隨機森林, 參數使用 Random Search 尋找
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
lr = LogisticRegression()
gdbt = GradientBoostingClassifier()
rf = RandomForestClassifier()

In [33]:
# 線性迴歸預測檔 (結果有部分隨機, 請以 Kaggle 計算的得分為準, 以下模型同理)
lr.fit(train_X, train_Y)
prediction = lr.predict_proba(test_X)[:,1]
print(lr.coef_)
print(lr.intercept_)
print(prediction)

[[-1.41835277  1.0851288   0.879144    0.19160798  0.20350307  0.29975946
   0.93927341  0.17441689  0.85266473]]
[-1.14332321]
[0.66820115 0.47453599 0.11145182 0.1880167  0.12163789 0.3738776
 0.18102285 0.10770781 0.87698378 0.19563876 0.21097615 0.12050474
 0.25509638 0.09473787 0.28151315 0.15756003 0.11543647 0.07210033
 0.09736114 0.09636134 0.14157432 0.11829125 0.09259644 0.07660337
 0.07479859 0.07819456 0.09422859 0.07211852 0.10161693 0.15641828
 0.07196809 0.14769809 0.11259093]


In [34]:
# 線性迴歸預測檔 (結果有部分隨機, 請以 Kaggle 計算的得分為準, 以下模型同理)
lr.fit(train_X, train_Y)
lr_pred = lr.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': lr_pred})
#sub['poi'] = sub['poi'].map(lambda x:1 if x>0.5 else 0) 

sub.to_csv('marathon_lr_TEST.csv', index=False) 

In [35]:
#### ** not finish **  檢查AUC
####
#### acc = metrics.accuracy_score(y_test, y_pred)
#### print("Accuracy: ", acc)
#### print("Feature importance: ", clf_bestparam.feature_importances_)

In [36]:
# 梯度提升機預測檔 
gdbt.fit(train_X, train_Y)
gdbt_pred = gdbt.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': gdbt_pred})
#sub['poi'] = sub['poi'].map(lambda x:1 if x>0.5 else 0) 
sub.to_csv('marathon_gdbt_TEST.csv', index=False)

In [37]:
# 隨機森林預測檔
rf.fit(train_X, train_Y)
rf_pred = rf.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': rf_pred})
#sub['poi'] = sub['poi'].map(lambda x:1 if x>0.5 else 0) 
sub.to_csv('marathon_rf_TEST.csv', index=False)

In [38]:
### ==== PRACTICE ENSEMBLE ==== Stacking & blending ### 

In [39]:
from mlxtend.classifier import StackingClassifier

meta_estimator = GradientBoostingClassifier(tol=100, subsample=0.70, n_estimators=50, 
                                           max_features='sqrt', max_depth=4, learning_rate=0.3)
stacking = StackingClassifier(classifiers=[lr,gdbt,rf],meta_classifier=meta_estimator)
stacking.fit(train_X, train_Y)
#stacking_pred = stacking.predict(test_X)
stacking_pred = stacking.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': stacking_pred})
sub.to_csv('marathon_stacking_TEST.csv', index=False)

In [40]:
blending_pred = lr_pred*0.25  + gdbt_pred*0.3 + rf_pred*0.45  #這兒可以調整參數 
sub = pd.DataFrame({'name': ids, 'poi': blending_pred})
sub.to_csv('marathon_blending_TEST.csv', index=False)